In [3]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

Hyper Parameters

In [4]:
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.003

MNIST Dataset

In [5]:
train_dataset = dsets.MNIST(
    root='../../data/', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = dsets.MNIST(
    root='../../data/', train=False, transform=transforms.ToTensor())

Data Loader (Input Pipeline)

In [6]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False)

BiRNN Model (Many-to-One)

In [7]:
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers,
            batch_first=True,
            bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # 2 for bidirection

    def forward(self, x):
        # Set initial states
        h0 = Variable(
            torch.zeros(self.num_layers * 2, x.size(0),
                        self.hidden_size)).cuda()  # 2 for bidirection
        c0 = Variable(
            torch.zeros(self.num_layers * 2, x.size(0),
                        self.hidden_size)).cuda()

        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))

        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])
        return out

In [8]:
rnn = BiRNN(input_size, hidden_size, num_layers, num_classes)
rnn.cuda()

BiRNN(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=10, bias=True)
)

Loss and Optimizer

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [10]:
# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, sequence_length, input_size)).cuda()
        labels = Variable(labels).cuda()

        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = rnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' %
                  (epoch + 1, num_epochs, i + 1,
                   len(train_dataset) // batch_size, loss.item()))

Epoch [1/2], Step [100/600], Loss: 0.7779
Epoch [1/2], Step [200/600], Loss: 0.2361
Epoch [1/2], Step [300/600], Loss: 0.1961
Epoch [1/2], Step [400/600], Loss: 0.2657
Epoch [1/2], Step [500/600], Loss: 0.1520
Epoch [1/2], Step [600/600], Loss: 0.0947
Epoch [2/2], Step [100/600], Loss: 0.1011
Epoch [2/2], Step [200/600], Loss: 0.0931
Epoch [2/2], Step [300/600], Loss: 0.0934
Epoch [2/2], Step [400/600], Loss: 0.1860
Epoch [2/2], Step [500/600], Loss: 0.0562
Epoch [2/2], Step [600/600], Loss: 0.0969


Test the Model

In [11]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, sequence_length, input_size)).cuda()
    outputs = rnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' %
      (100 * correct / total))

Test Accuracy of the model on the 10000 test images: 97 %


Save the Model

In [12]:
torch.save(rnn.state_dict(), 'rnn.pkl')